In [ ]:
import pandas as pd
from plot_StrengthSurface import filter_data
import local_config
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib.cm as cm

: 

In [ ]:
class Surface:
    def __init__(self, points):
        """
        This class defines a single strength surface and can fit a Drucker-Prager model to it
        
        df (list[DataPoint]): list of datapoints for the surface
        """

        self.points = points
        self.seed = self.check_seed()
        self.alpha = None
        self.k = None
        self.fit_result = None

    def check_seed(self):
        """
        Check to make sure all of our random seeds match up, returns the seed if it does match
        """

        seed = self.points[0].df["Defect Random Seed"]
        for p in self.points:
            if p.df["Defect Random Seed"] != seed:
                raise ValueError("Random Seeds do not match up in this surface!")
        return seed

    def loss(self, params):
        """
        Loss function to minimize: sum of squared Drucker-Prager residuals.

        params: [alpha, k]
        """
        alpha, k = params
        residuals = []
        for point in self.points:
            i1, j2 = point.calculate_invariants()
            residual = np.sqrt(j2) + alpha * i1 - k
            residuals.append(residual ** 2)
        return sum(residuals)

    def fit_drucker_prager(self):
        """
        Fit Drucker-Prager parameters (alpha, k) to the current surface by minimizing the least squares loss.
        Stores the optimized values in self.alpha and self.k.
        """
        try:
            result = minimize(self.loss, x0=[0.0, 1.0])
            self.fit_result = result 
            if result.success or "precision loss" in result.message:
                self.alpha, self.k = result.x
            else:
                raise RuntimeError(f"Minimization failed: {result.message}")

        except RuntimeError as e:
            print(f"Warning: Seed {int(self.seed)} fit failed. {e}")
            self.alpha, self.k = np.nan, np.nan

    def compute_loss_statistics(self):
        if self.alpha is np.nan or self.k is np.nan:
            return {
                "total_loss": np.nan,
                "mse": np.nan,
                "rmse": np.nan,
                "max_residual": np.nan
            }
        
        residuals = []
        for point in self.points:
            i1, j2 = point.calculate_invariants()
            r = np.sqrt(j2) + self.alpha * i1 - self.k
            residuals.append(r)

        residuals = np.array(residuals)
        n = len(residuals)
        total_loss = np.sum(residuals**2)
        mse = total_loss / n
        rmse = np.sqrt(mse)
        max_residual = np.max(np.abs(residuals))

        return {
            "total_loss": total_loss,
            "mse": mse,
            "rmse": rmse,
            "max_residual": max_residual
        }
    
    def plot_loss_landscape(self, alpha_range=(-0.1, 0.1), k_range=(25, 50), res=100):

        alphas = np.linspace(*alpha_range, res)
        ks = np.linspace(*k_range, res)
        A, K = np.meshgrid(alphas, ks)
        Loss = np.zeros_like(A)

        for i in range(res):
            for j in range(res):
                Loss[i, j] = self.loss([A[i, j], K[i, j]])

        plt.figure(figsize=(8, 6))
        cp = plt.contourf(A, K, Loss, levels=50, cmap='viridis')
        plt.colorbar(cp, label='Loss')
        plt.xlabel(r"$\alpha$")
        plt.ylabel(r"$k$")
        plt.title(f"Loss Landscape for Seed {self.seed}")

        # Plot optimizer's best guess
        if self.fit_result is not None:
            a, k = self.fit_result.x
            plt.plot(a, k, 'ro', label="Optimizer guess")
            plt.legend()

        plt.savefig(f"Loss_Landscape_{self.seed}.png")



In [ ]:
class DataPoint:
    def __init__(self, df):
        """
        This class defines a single point in the strength surface
        
        df (pandas dataframe): dataframe for one single datapoint in the big dataframe (one row long)
        """
        self.df = df

    def calculate_invariants(self):
        # we only store the principal stresses, so the stress tensor is diagonal by default
        sigma = np.diagflat([self.df["Strength_1"], self.df["Strength_2"], self.df["Strength_3"]])
        i1 = np.trace(sigma)  # tr(sigma)
        dev_stress = sigma - (1 / 3) * i1 * np.identity(3)
        dev2 = dev_stress @ dev_stress
        j2 = 0.5 * np.trace(dev2)
        return i1, j2
 

In [ ]:
def plot_surface_fit(surface, resolution=1000):
    alpha = surface.alpha
    k = surface.k

    # Set plot range around your data
    sig1_vals = [dp.df["Strength_1"] for dp in surface.points]
    sig2_vals = [dp.df["Strength_2"] for dp in surface.points]
    min_sig, max_sig = min(sig1_vals + sig2_vals), max(sig1_vals + sig2_vals)
    grid = np.linspace(min_sig * 1.1, max_sig * 1.1, resolution)

    # Create sigma1, sigma2 grid
    sig1, sig2 = np.meshgrid(grid, grid)
    sig3 = np.zeros_like(sig1)  

    # Compute I1 and sqrt(J2)
    i1 = sig1 + sig2 + sig3
    mean_stress = i1 / 3
    dev_xx = sig1 - mean_stress
    dev_yy = sig2 - mean_stress
    dev_zz = sig3 - mean_stress

    j2 = 0.5 * (dev_xx**2 + dev_yy**2 + dev_zz**2)
    sqrtJ2 = np.sqrt(j2)

    # Evaluate DP function
    F = sqrtJ2 + alpha * i1 - k

    plt.figure(figsize=(8, 8))

    # Plot contour where f = 0 (the strength boundary)
    plt.contour(sig1, sig2, F, levels=[0], colors="red", linewidths=2)
    plt.plot([], [], color="red", label="DP surface")  # for legend (cs.collections is not working)

    # Plot data points
    plt.scatter(sig1_vals, sig2_vals, color="blue", label="MD failure points")
    plt.scatter(sig2_vals, sig1_vals, color="blue")

    plt.plot([-50, 130], [0, 0], color='black')
    plt.plot([0, 0], [-50, 130], color='black')

    plt.xlabel(r"$\sigma_1$")
    plt.ylabel(r"$\sigma_2$")

    plt.xlim(-15, 100)
    plt.ylim(-15, 100)

    plt.title(f"Fitted Drucker-Prager Surface (Seed {int(surface.seed)})")
    plt.legend()

    plt.savefig(f'{local_config.DATA_DIR}/defected_data/plots/DP_fitted_{int(surface.seed)}.png')
    plt.close()


def plot_all_surfaces(surfaces, resolution=1000, showlabels=False):
    # Set global grid range
    all_sig_vals = [dp.df["Strength_1"] for s in surfaces for dp in s.points] + \
                   [dp.df["Strength_2"] for s in surfaces for dp in s.points]
    min_sig, max_sig = min(all_sig_vals), max(all_sig_vals)
    padding = 0.2 * (max_sig - min_sig)
    grid = np.linspace(min_sig - padding, max_sig + padding, resolution)

    sig1, sig2 = np.meshgrid(grid, grid)
    sig3 = np.zeros_like(sig1)

    fig, ax = plt.subplots(figsize=(8, 8))

    # Cycle through colors for each seed
    colors = cm.hsv(np.linspace(0, 1, len(surfaces)))

    for surface, color in zip(surfaces, colors):
        alpha = surface.alpha
        k = surface.k

        i1 = sig1 + sig2 + sig3
        mean_stress = i1 / 3
        dev_xx = sig1 - mean_stress
        dev_yy = sig2 - mean_stress
        dev_zz = sig3 - mean_stress
        j2 = 0.5 * (dev_xx**2 + dev_yy**2 + dev_zz**2)
        sqrtJ2 = np.sqrt(j2)
        F = sqrtJ2 + alpha * i1 - k

        # Plot contour
        ax.contour(sig1, sig2, F, levels=[0], colors=[color], linewidths=1.5)
        if showlabels:
            ax.plot([], [], color=color, label=f"Seed {int(surface.seed)}")

    ax.plot([-50, 130], [0, 0], color='black')
    ax.plot([0, 0], [-50, 130], color='black')

    ax.set_xlim(-15, 100)
    ax.set_ylim(-15, 100)
    ax.set_xlabel(r"$\sigma_1$")
    ax.set_ylabel(r"$\sigma_2$")
    ax.set_title("DP Surfaces Overlayed by Random Seed")
    if showlabels:
        ax.legend()

    plt.savefig(f'{local_config.DATA_DIR}/defected_data/plots/DP_overlay_all_seeds.png')
    plt.close()
